# Experiment 041: Circuit Complexity Boundary

**Question:** Do fixed-width networks have hard performance boundaries determined by SOS width?

**Hypothesis:** For B=8, beta=2: max solvable k = (8/2) - 1 = 3. Tasks with k>3 should fail.

**Key Prediction:** Sharp accuracy drop at k=4.

In [ ]:
!pip install transformers torch matplotlib -q

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

## 1. Load Model

In [ ]:
MODEL_NAME = "gpt2"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME).to(device)
model.eval()

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

print(f"Loaded {MODEL_NAME}")

## 2. Circuit Complexity Theory

```
REQUIRED BREADTH = (k + 1) x beta

For AKIRA-like architecture:
  B = 8 (breadth)
  beta = 2 (predicate arity)
  
Max solvable k = (B/beta) - 1 = (8/2) - 1 = 3

Prediction:
  k <= 3: Can solve
  k > 3: Cannot solve (mathematically impossible)
```

## 3. Generate Tasks with Controlled SOS Width

In [ ]:
def generate_k1_tasks(n=20):
    """k=1: Track single object property."""
    colors = ['red', 'blue', 'green', 'yellow']
    objects = ['ball', 'box', 'cup']
    tasks = []
    for i in range(n):
        c = colors[i % len(colors)]
        o = objects[i % len(objects)]
        tasks.append({
            'prompt': f"There is a {c} {o}. What color is the {o}?",
            'answer': c
        })
    return tasks

def generate_k2_tasks(n=20):
    """k=2: Track object + one relationship."""
    tasks = []
    for i in range(n):
        tasks.append({
            'prompt': f"A is on the table. B is on A. Where is B?",
            'answer': 'A'
        })
    return tasks

def generate_k3_tasks(n=20):
    """k=3: Track object + two relationships."""
    tasks = []
    for i in range(n):
        tasks.append({
            'prompt': f"A is on the table. B is on A. C is on B. What is on top?",
            'answer': 'C'
        })
    return tasks

def generate_k4_tasks(n=20):
    """k=4: Track object + three relationships (beyond boundary)."""
    tasks = []
    for i in range(n):
        tasks.append({
            'prompt': f"A is on the table. B is on A. C is on B. D is on C. "
                     f"If B is removed, what falls?",
            'answer': 'C and D'  # or just 'C'
        })
    return tasks

def generate_k5_tasks(n=20):
    """k=5: Well beyond boundary."""
    tasks = []
    for i in range(n):
        tasks.append({
            'prompt': f"A is on table. B is on A. C is on B. D is on C. E is on D. "
                     f"List the order from bottom to top.",
            'answer': 'A, B, C, D, E'
        })
    return tasks

# Generate all tasks
N_PER_K = 20
all_tasks = {
    1: generate_k1_tasks(N_PER_K),
    2: generate_k2_tasks(N_PER_K),
    3: generate_k3_tasks(N_PER_K),
    4: generate_k4_tasks(N_PER_K),
    5: generate_k5_tasks(N_PER_K)
}

print("Generated tasks:")
for k, tasks in all_tasks.items():
    print(f"  k={k}: {len(tasks)} tasks")
    print(f"    Example: {tasks[0]['prompt'][:60]}...")

## 4. Evaluate Model on Tasks

In [ ]:
def evaluate_task(prompt, expected):
    """Evaluate model on a single task."""
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=20,
            temperature=0.1,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    generated = response[len(prompt):].strip().lower()
    expected_lower = expected.lower()
    
    # Check if answer is in response
    correct = expected_lower in generated or generated.startswith(expected_lower)
    
    return {
        'generated': generated[:50],
        'expected': expected,
        'correct': correct
    }

# Test
test_result = evaluate_task("The sky is blue. What color is the sky?", "blue")
print(f"Test: {test_result}")

In [ ]:
# Evaluate all k levels
results = {}

for k, tasks in all_tasks.items():
    print(f"\nEvaluating k={k}...")
    correct = 0
    task_results = []
    
    for task in tasks:
        result = evaluate_task(task['prompt'], task['answer'])
        task_results.append(result)
        if result['correct']:
            correct += 1
    
    accuracy = correct / len(tasks)
    results[k] = {
        'accuracy': accuracy,
        'correct': correct,
        'total': len(tasks),
        'details': task_results
    }
    
    print(f"  Accuracy: {accuracy:.1%} ({correct}/{len(tasks)})")

print("\n" + "="*50)

## 5. Analyze Boundary

In [ ]:
# Extract accuracies
k_values = sorted(results.keys())
accuracies = [results[k]['accuracy'] for k in k_values]

# Compute drops
drops = []
for i in range(len(accuracies) - 1):
    drop = accuracies[i] - accuracies[i+1]
    drops.append({
        'from_k': k_values[i],
        'to_k': k_values[i+1],
        'drop': drop
    })

print("ACCURACY DROPS:")
for d in drops:
    significance = "SIGNIFICANT" if d['drop'] > 0.3 else "minor"
    print(f"  k={d['from_k']} -> k={d['to_k']}: {d['drop']:.1%} [{significance}]")

# Find boundary
max_drop = max(drops, key=lambda x: x['drop'])
observed_boundary = max_drop['to_k'] if max_drop['drop'] > 0.2 else None
predicted_boundary = 4  # For B=8, beta=2: max k = 3, so boundary at k=4

print(f"\nObserved boundary: k = {observed_boundary}")
print(f"Predicted boundary: k = {predicted_boundary}")

## 6. Visualization

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Plot 1: Accuracy by k
ax1 = axes[0]
colors = ['green' if a > 0.7 else 'orange' if a > 0.4 else 'red' for a in accuracies]
bars = ax1.bar(k_values, accuracies, color=colors, alpha=0.7, edgecolor='black')

# Add boundary line
ax1.axvline(x=predicted_boundary - 0.5, color='red', linestyle='--', 
            linewidth=2, label=f'Predicted boundary (k={predicted_boundary})')
ax1.axhline(y=0.5, color='gray', linestyle=':', alpha=0.5)

ax1.set_xlabel('SOS Width (k)', fontsize=12)
ax1.set_ylabel('Accuracy', fontsize=12)
ax1.set_title('Accuracy by SOS Width', fontsize=14)
ax1.set_ylim(0, 1.1)
ax1.set_xticks(k_values)
ax1.legend()

# Add value labels
for bar, acc in zip(bars, accuracies):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.02, 
             f'{acc:.0%}', ha='center', fontsize=10)

# Plot 2: Drop magnitude
ax2 = axes[1]
labels = [f"k={d['from_k']}->k={d['to_k']}" for d in drops]
drop_values = [d['drop'] for d in drops]
colors = ['red' if d > 0.3 else 'orange' if d > 0.1 else 'green' for d in drop_values]

ax2.bar(range(len(drops)), drop_values, color=colors, alpha=0.7, edgecolor='black')
ax2.axhline(y=0.3, color='red', linestyle='--', label='Significant threshold')
ax2.set_xticks(range(len(drops)))
ax2.set_xticklabels(labels)
ax2.set_xlabel('Transition', fontsize=12)
ax2.set_ylabel('Accuracy Drop', fontsize=12)
ax2.set_title('Accuracy Drop at Each Transition', fontsize=14)
ax2.legend()

plt.tight_layout()
plt.show()

## 7. Summary

In [ ]:
print("="*60)
print("EXPERIMENT 041 SUMMARY")
print("="*60)

print("\n1. THEORETICAL PREDICTION:")
print("   Breadth B = 8")
print("   Predicate arity beta = 2")
print("   Max solvable k = (B/beta) - 1 = 3")
print("   Boundary at k = 4")

print("\n2. OBSERVED RESULTS:")
for k in k_values:
    acc = results[k]['accuracy']
    status = "PASS" if acc > 0.6 else "FAIL"
    print(f"   k = {k}: {acc:.0%} [{status}]")

print(f"\n3. BOUNDARY ANALYSIS:")
print(f"   Observed boundary: k = {observed_boundary}")
print(f"   Predicted boundary: k = {predicted_boundary}")
print(f"   Max drop: {max_drop['drop']:.1%} at k={max_drop['from_k']}->{max_drop['to_k']}")

print("\n4. VERDICT:")

# Check criteria
acc_k3 = results.get(3, {}).get('accuracy', 0)
acc_k4 = results.get(4, {}).get('accuracy', 1)
sharp_drop = acc_k3 - acc_k4 > 0.25
boundary_match = observed_boundary == predicted_boundary

if sharp_drop:
    print("   HYPOTHESIS SUPPORTED")
    print(f"   - Sharp drop at k=4: {acc_k3:.0%} -> {acc_k4:.0%}")
    print("   - Fixed-width networks have complexity limits")
    print("   - Circuit complexity theory validated")
else:
    print("   HYPOTHESIS NOT SUPPORTED")
    print(f"   - No sharp drop: k=3 ({acc_k3:.0%}) vs k=4 ({acc_k4:.0%})")
    print("   - Model may use different strategies")
    print("   - Or tasks don't properly isolate complexity")